In [1]:
## Purpose: Scrape wikipedia for a listing of IPOs from 2000 to 2019. To do so 
## I have created a bot that will click on each company on the given page and scrape the necessary data.
## I will loop through the pages for the years 2000 to 2019.

## I have named the bot "Walter the Wikipedia Web Scraper" after a Corgi I met hiking.

In [1]:
## import web scraping packages
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

## import web surfing packages
import selenium 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np
from time import sleep

## Walter the Wikipedia Web Scraping Robot

In [9]:
## create Walter the Wikipedia Web Scraper in a method

def walter(year):
    ## create html and open for bs 
    html = 'https://en.wikipedia.org/wiki/Category:' + str(year) + '_initial_public_offerings'
    html_open = urlopen(html)
    
    ## initialize webdriver and go to html site
    ## reference driver
    #global driver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(html)

    ## get the html source of the current page (will be the page listing all the IPOs for that year)
    soup_year = bs(html_open)

    ## get container of all the companies. Under html "div"
    container = soup_year.find("div", {"class": "mw-category"})

    ## find all the companies ('c') on page
    c1 = container.find_all("li")

    ## loop through c1 and use 'get_text' function to put all companies into a list ('c2')
    c2 = []
    
    ## create a list of dictionaries to keep the data in
    lst_dict = []
    
    ## create a value to keep track of companies passed in cy. Create value to track exceptions. Reference global passed value
    passed_cy = []
    exceptions = 0
    global passed

    for i in range(0, len(c1)):
        name = c1[i].get_text()
        c2.append(name)

    ## loop through company list (c2) and click on each company in list to visit company's wikipedia page
    for x in c2:
        ## initialize variables of interest with np.NaN. If variable is not available it 
        ## will be still be included in df.
        industry = np.NaN
        founded = np.NaN
        headquarters = np.NaN
        ticker = np.NaN
        
        try:
            ## click on company link
            driver.find_element_by_link_text(x).click()
            ## slow down crawler by 5 seconds
            sleep(5)
            try:
                ## get html source of page
                soup_company = bs(driver.page_source, 'html.parser')  
                ## get company's info table
                table = soup_company.find("table", {"class": "infobox vcard"})
                ## get all rows from the table
                rows = table.find_all("tr")

                ##create values list to keep track of points of interest
                vals = [x, year]

                ## loop through all rows and grab 'industry', 'founded', 'hq'
                ## in info unavailable, variable will be np.NaN
                for i in range(0, len(rows)):
                    ## use try for exception handling
                    try:
                        ## get header of variable and match to point of interest
                        header = rows[i].find("th").get_text()

                        if header == 'Industry':
                            industry = rows[i].find("td").get_text()
                        elif header == 'Founded':
                            founded = rows[i].find("td").get_text()
                        elif header == 'Headquarters':
                            headquarters = rows[i].find("td").get_text()
                        elif header == 'Traded&nbsp;as':
                            ticker = rows[i].find("td").get_text()
                        else:
                            ## if row does not match then skip
                            pass

                    except:
                        pass
            except:
                passed.append(x)
                exceptions += 1
                pass
        except:
            passed.append(x)
            exceptions += 1
            pass

        ## add variables to vals list and append to running list dictionary
        vals.append(ticker)
        vals.append(industry)
        vals.append(founded)
        vals.append(headquarters)
        lst_dict.append(dict(zip(cols, vals))) 

        ## go back one page to listing of companies
        driver.execute_script("window.history.go(-1)")
        
        ## add in delay before going to next company
        sleep(5)
        
    ## add in message that year is complete and how many companies were passed
    print("Walter complete for year " + str(year) + ". Number of companies passed was: " + str(exceptions))
    
    ## close web browser
    #driver.close()
    
    ## quit web browser
    driver.quit()
    
    return lst_dict

  

In [10]:
## Create a list of the years of interest (2000 to 2019). This will be used to click through the 
## wikipedia pages. 

years = list(np.arange(2000,2020))

## Create list of columns for the datapoints of interest - use in creating dataframe
cols = ['Company', 'Year', 'Ticker', 'Industry', 'Founded', 'Headquarters']

## Create a dataframe
df = pd.DataFrame(columns = cols)

## Create a list to keep track of companies that were passed
passed = []

In [11]:
## Create a for loop calling the web scraper for each html

## initialize webdriver 
#driver = webdriver.Chrome()

for year in years:
    ## call web scraper
    output = walter(year)
    ## append output to dataframe
    df = df.append(output, ignore_index = True)
    ## wait one minute before repeating
    sleep(60)
    
## save dataframe to csv
df.to_csv('ipoCompanies.csv', index = False)

## end chromdriver
#driver.quit()

[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - File found in cache by path [C:\Users\nicks\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


Walter complete for year 2000. Number of companies passed was: 0


[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - File found in cache by path [C:\Users\nicks\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


Walter complete for year 2001. Number of companies passed was: 0


[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - File found in cache by path [C:\Users\nicks\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


...

Walter complete for year 2017. Number of companies passed was: 3


[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - File found in cache by path [C:\Users\nicks\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


Walter complete for year 2018. Number of companies passed was: 1


[WDM] - Looking for [chromedriver 83.0.4103.39 win32] driver in cache 
[WDM] - File found in cache by path [C:\Users\nicks\.wdm\drivers\chromedriver\83.0.4103.39\win32\chromedriver.exe]


Walter complete for year 2019. Number of companies passed was: 0


In [12]:
df

,Company,Year,Ticker,Industry,Founded,Headquarters
0,360networks,2000,NaN,Telecommunications,"February 5, 1998; 22 years ago (1998-02-05)","Seattle, Washington"
1,Airspan Networks,2000,NaN,Telecommunications equipment,"January 30, 1998; 22 years ago (1998-01-30)","Boca Raton, Florida"
2,Array BioPharma,2000,NaN,OncologyMedication,1998; 22 years ago (1998),"Boulder, Colorado, United States of America"
3,AT&T Wireless Services,2000,NaN,Wireless telecommunications,1987; 33 years ago (1987) (as McCaw Cellular C...,"Redmond, Washington, U.S."
4,Beasley Broadcast Group,2000,NaN,Advertising; Radio broadcasting,"Benson, North Carolina (1961)","Naples, Florida, United States"
5,Caldera International,2000,NaN,Operating system software,"Orem, Utah (1998)","Orem, Utah, United States"
6,California Pizza Kitchen,2000,NaN,Food and beverage,1985; 35 years ago (1985),"Los Angeles, California, United States"
7,Cepheid Inc,2000,NaN,BiotechnologyMedical devices,March 1996; 24 years ago (1996-03),"Sunnyvale, California, US"
8,Charles River Laboratories,2000,NaN,Pharmaceutical/medical devices,1947; 73 years ago (1947),"Wilmington, Massachusetts"
9,Community Health Systems,2000,NaN,Health care,1985,"Franklin, Tennessee, United States"


In [42]:
## save dataframe to csv
df.to_csv('ipoCompanies.csv', index = False)

In [20]:
## save passed companies to csv
passed = pd.Series(passed)
passed = pd.DataFrame(passed)
passed.to_csv('passed.csv', index = False)

In [21]:
driver.quit()

In [ ]:
## taskkill /F /IM ChromeDriver.exe